In [ ]:
from datasets import load_dataset

ds = load_dataset('ape-ds')

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = ""
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained('my-tokenizer')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
def make_translations(example):
    input_ids = tokenizer("translate Tibetan to English: "+example['bo'], return_tensors="pt").to('cuda:0').input_ids
    outputs = model.generate(input_ids)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['for-post-edit'] = translation
    return example

ds = ds.map(make_translations)

In [ ]:
ds.save_to_disk('ape-ds')